# Core10_09 — Counterfactual Experiment Matrix

## 목적
단일 시뮬레이션이 아니라, **정책 × 시나리오 매트릭스**로
운영 결과를 비교한다.

- Baseline A: Random allocation
- Baseline B: Score-based allocation (의도적으로 금지된 기준)
- Proposed: State / trajectory-aware policy (Core10)

## 시나리오 축
- Cold: 저스트레스 / 저드리프트
- Hot: 고스트레스 / 누적 드리프트
- Oscillation-prone: 토글 민감 환경

## 산출물
- experiment_plan_table.csv
- (후속) 각 실험별 core10_07 rollout log

In [2]:
from pathlib import Path
import json
import itertools
import numpy as np
import pandas as pd

ART_CORE10 = Path("../artifact/core10")
ART_CORE10.mkdir(parents=True, exist_ok=True)

POLICY_PATH = ART_CORE10 / "core10_06_allocation_policy.json"
SCORES_PATH = ART_CORE10 / "core10_04_survivability_scores.csv"

assert POLICY_PATH.exists(), "Missing core10_06_allocation_policy.json"
assert SCORES_PATH.exists(), "Missing core10_04_survivability_scores.csv"

policy = json.loads(POLICY_PATH.read_text(encoding="utf-8"))
scores = pd.read_csv(SCORES_PATH)

print("policy_id:", policy.get("policy_id"))
print("n_candidates:", len(scores))

policy_id: core10_06_allocation_policy_v1
n_candidates: 19


### Experiment Axes

**Allocation Policy Axis**
- RANDOM: 무작위 선택 (baseline)
- SCORE: survivability score 최대화 (금지 기준)
- STATEFUL: Core10 정책 (hazard + TTL + cooldown)

**Scenario Axis**
- COLD: drift ↓, noise ↓
- HOT: drift ↑, 누적 stress ↑
- OSCILLATION: toggle 민감도 ↑

In [3]:
POLICY_AXIS = {
    "BASELINE_RANDOM": {
        "description": "Random allocation baseline",
        "type": "random",
    },
    "BASELINE_SCORE": {
        "description": "Score-based allocation (forbidden baseline)",
        "type": "score_max",
    },
    "PROPOSED_STATEFUL": {
        "description": "Core10 state/trajectory-aware policy",
        "type": "stateful",
        "policy_id": policy.get("policy_id"),
    },
} # Policy Axis Definition

In [4]:
SCENARIO_AXIS = {
    "COLD": {
        "drift_enabled": False,
        "drift_per_step": 0.0,
        "noise_scale": 0.01,
        "toggle_sensitivity": 0.5,
    },
    "HOT": {
        "drift_enabled": True,
        "drift_per_step": 0.03,
        "noise_scale": 0.03,
        "toggle_sensitivity": 1.0,
    },
    "OSCILLATION": {
        "drift_enabled": True,
        "drift_per_step": 0.015,
        "noise_scale": 0.02,
        "toggle_sensitivity": 1.8,
    },
} # Scenario Axis Definition

In [5]:
# Global reproducibility contract
BASE_SEED = 20260109

def derive_seed(*keys):
    """
    Deterministic seed derivation from experiment identifiers
    """
    h = hash("_".join(map(str, keys)))
    return abs(h) % (2**32) # Seed Policy

In [6]:
rows = []

EXP_ID = 0

for policy_key, scenario_key in itertools.product(POLICY_AXIS, SCENARIO_AXIS):
    seed = derive_seed(BASE_SEED, policy_key, scenario_key)

    rows.append({
        "experiment_id": f"EXP_{EXP_ID:03d}",
        "policy_key": policy_key,
        "policy_type": POLICY_AXIS[policy_key]["type"],
        "policy_description": POLICY_AXIS[policy_key]["description"],

        "scenario_key": scenario_key,
        "scenario_config": json.dumps(SCENARIO_AXIS[scenario_key]),

        "seed": seed,
        "n_steps": 60,               # shared contract
        "ttl_steps": 8,
        "cooldown_steps": policy.get("eligibility", {}).get("cooldown", {}).get("cooldown_steps", 0),
    })
    EXP_ID += 1

experiment_plan = pd.DataFrame(rows)
experiment_plan # Experiment Matrix Construction

,experiment_id,policy_key,policy_type,policy_description,scenario_key,scenario_config,seed,n_steps,ttl_steps,cooldown_steps
0,EXP_000,BASELINE_RANDOM,random,Random allocation baseline,COLD,"{""drift_enabled"": false, ""drift_per_step"": 0.0...",1458256497,60,8,3
1,EXP_001,BASELINE_RANDOM,random,Random allocation baseline,HOT,"{""drift_enabled"": true, ""drift_per_step"": 0.03...",2865804044,60,8,3
2,EXP_002,BASELINE_RANDOM,random,Random allocation baseline,OSCILLATION,"{""drift_enabled"": true, ""drift_per_step"": 0.01...",1780719939,60,8,3
3,EXP_003,BASELINE_SCORE,score_max,Score-based allocation (forbidden baseline),COLD,"{""drift_enabled"": false, ""drift_per_step"": 0.0...",3757395954,60,8,3
4,EXP_004,BASELINE_SCORE,score_max,Score-based allocation (forbidden baseline),HOT,"{""drift_enabled"": true, ""drift_per_step"": 0.03...",4140186586,60,8,3
5,EXP_005,BASELINE_SCORE,score_max,Score-based allocation (forbidden baseline),OSCILLATION,"{""drift_enabled"": true, ""drift_per_step"": 0.01...",4153684021,60,8,3
6,EXP_006,PROPOSED_STATEFUL,stateful,Core10 state/trajectory-aware policy,COLD,"{""drift_enabled"": false, ""drift_per_step"": 0.0...",2866911993,60,8,3
7,EXP_007,PROPOSED_STATEFUL,stateful,Core10 state/trajectory-aware policy,HOT,"{""drift_enabled"": true, ""drift_per_step"": 0.03...",2411503323,60,8,3
8,EXP_008,PROPOSED_STATEFUL,stateful,Core10 state/trajectory-aware policy,OSCILLATION,"{""drift_enabled"": true, ""drift_per_step"": 0.01...",899820554,60,8,3


### Interpretation

Each row = **one fully specified counterfactual world**

- Same candidate pool
- Same initial state
- Different:
  - allocation rule
  - environmental stress dynamics
  - deterministic seed

This table is the **single source of truth** for:
- batch simulation
- parallel rollout
- audit & reproducibility

In [7]:
OUT_PATH = ART_CORE10 / "core10_09_experiment_plan_table.csv"
experiment_plan.to_csv(OUT_PATH, index=False)

print("✅ Exported experiment plan:")
print(OUT_PATH.resolve())
print("n_experiments:", len(experiment_plan)) # Export Experiment Plan

✅ Exported experiment plan:
/Users/mac/Desktop/De/Developability_Data/core/artifact/core10/core10_09_experiment_plan_table.csv
n_experiments: 9


## Execution Contract (Next Notebook)

For each row in `experiment_plan_table.csv`:

1. Load row
2. Set RNG seed = `seed`
3. Inject scenario parameters into Core10_07 dynamics
4. Select allocation engine:
   - RANDOM
   - SCORE_MAX
   - STATEFUL (Core10)
5. Run scheduler
6. Save:
   - rollout log
   - summary metrics
   - experiment_id tagged

➡️ This becomes `core10_10_batch_runner.ipynb`